In [2]:
import warnings
# warnings. simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
from tabulate import tabulate

In [123]:
data = GoldenCrossverSignal('RELIANCE')
data.to_csv('data.csv')
len(data)

9

In [99]:
required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]

In [100]:
data[data['Position'] == 'Sell'].index[-1]

Timestamp('2023-08-01 00:00:00')

In [3]:

# Name, Entry TIme, Entry PRice, QTY, Exit Time, Exit Price

class Backtest:
    def __init__(self):
        self.columns = ['Equity Name', 'Trade', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'Quantity', 'Position Size', 'PNL', '% PNL']
        self.backtesting = pd.DataFrame(columns=self.columns)

    def buy(self, equity_name, entry_time, entry_price, qty):
        self.trade_log = dict(zip(self.columns, [None] * len(self.columns)))
        self.trade_log['Trade'] = 'Long Open'
        self.trade_log['Quantity'] = qty
        self.trade_log['Position Size'] = round(self.trade_log['Quantity'] * entry_price, 3)
        self.trade_log['Equity Name'] = equity_name
        self.trade_log['Entry Time'] = entry_time
        self.trade_log['Entry Price'] = round(entry_price, 2)

    def sell(self, exit_time, exit_price, exit_type, charge):
        self.trade_log['Trade'] = 'Long Closed'
        self.trade_log['Exit Time'] = exit_time
        self.trade_log['Exit Price'] = round(exit_price, 2)
        self.trade_log['Exit Type'] = exit_type
        self.trade_log['PNL'] = round((self.trade_log['Exit Price'] - self.trade_log['Entry Price']) * self.trade_log['Quantity'] - charge, 3)
        self.trade_log['% PNL'] = round((self.trade_log['PNL'] / self.trade_log['Position Size']) * 100, 3)
        self.trade_log['Holding Period'] = exit_time - self.trade_log['Entry Time']
        # self.backtesting = self.backtesting.append(self.trade_log, ignore_index=True)
        self.backtesting.loc[len(self.backtesting)] = self.trade_log
    def stats(self):
        df = self.backtesting
        parameters = ['Total Trade Scripts', 'Total Trade', 'PNL',  'Winners', 'Losers', 'Win Ratio','Total Profit', 'Total Loss', 'Average Loss per Trade', 'Average Profit per Trade', 'Average PNL Per Trade', 'Risk Reward']
        total_traded_scripts = len(df['Equity Name'].unique())
        total_trade = len(df.index)
        pnl = df.PNL.sum()
        winners = len(df[df.PNL > 0])
        loosers = len(df[df.PNL <= 0])
        win_ratio = str(round((winners/total_trade) * 100, 2)) + '%'
        total_profit = round(df[df.PNL > 0].PNL.sum(), 2)
        total_loss  = round(df[df.PNL <= 0].PNL.sum(), 2)
        average_loss_per_trade = round(total_loss/loosers, 2)
        average_profit_per_trade = round(total_profit/winners, 2)
        average_pnl_per_trade = round(pnl/total_trade, 2)
        risk_reward = f'1:{-1 * round(average_profit_per_trade/average_loss_per_trade, 2)}'
        data_points = [total_traded_scripts,total_trade,pnl,winners, loosers, win_ratio, total_profit, total_loss, average_loss_per_trade, average_profit_per_trade, average_pnl_per_trade, risk_reward]
        data = list(zip(parameters,data_points ))
        print(tabulate(data, ['Parameters', 'Values'], tablefmt='psql'))
        

In [4]:
from datetime import datetime, timedelta
seven_days_ago = datetime.now() - timedelta(days=30)
today_date = datetime.now()- timedelta(days=7)
stocks_to_buy =[]
def GoldenCrossverSignal(name,x,y):
    path = f'./Data/{name}.csv'
    data = pd.read_csv(path, parse_dates=['Date'], index_col='Date') #index_col='Date'
    # data['Date'] = pd.to_datetime(data['Date'])
    
    data['Prev_Close'] = data.Close.shift(1)
    data['20_SMA'] = data.Prev_Close.rolling(window=20, min_periods=1).mean()
    data['50_SMA'] = data.Prev_Close.rolling(window=50, min_periods=1).mean()
    data['20_SMA_diff'] =  data['20_SMA'] - data['20_SMA'].shift(7)
    data['50_SMA_diff'] = data['50_SMA'].shift(-14) - data['50_SMA']
    data = data.iloc[-x:y]
    data['Signal'] = 0
    data['Signal'] = np.where((data['50_SMA_diff'] > 0), 1, 0)
    data['Position'] = data.Signal.diff()
    df_pos = data[(data['Position'] == 1) | (data['Position'] == -1)].copy()
    df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x == 1 else 'Sell')
    # print(df_pos.columns)
    # Filter the DataFrame based on the conditions
    # try:
    #     filtered_df = df_pos.loc[(df_pos["Date"] >= seven_days_ago)]
    #     if len(filtered_df) : 
    #         # print(filtered_df['Position'].iat[-1] )
    #         if   filtered_df['Position'].iat[-1] == "Buy":
    #             stocks_to_buy.append(name)
    #             sig_date = filtered_df['Date'].iloc[0]
    #             z= sig_date - pd.Timedelta(days=0)
    #             old_price = filtered_df.loc[filtered_df['Date'] == z]['Close'].iloc[0]
    #             cur_price = data.loc[data['Date'] >= today_date]['Close'].iloc[0]
    #             perc = ((cur_price-old_price)/old_price)*100
    #             print(name,perc,sig_date)
    # except Exception as e:
    #     print(f"error {name} , {e}")
    

    return  df_pos
sty='3PLAND'
GoldenCrossverSignal(sty,0,-1)

,Open,High,Low,Close,Adj Close,Volume,Prev_Close,20_SMA,50_SMA,20_SMA_diff,50_SMA_diff,Signal,Position
Date,,,,,,,,,,,,,
2014-02-28,6.900000,6.900000,6.900000,6.900000,6.900000,0,6.900000,6.9200,7.093333,-0.1400,5.530296e-03,1,Buy
2014-03-12,7.050000,7.150000,7.050000,7.050000,7.050000,992,7.500000,7.0025,7.106579,0.0875,-5.789099e-04,0,Sell
2014-03-13,7.050000,7.050000,7.050000,7.050000,7.050000,200,7.050000,7.0100,7.105128,0.1000,3.871830e-03,1,Buy
2014-09-04,13.200000,13.200000,13.200000,13.200000,13.200000,100,12.800000,12.9625,13.228000,-0.8725,-4.300001e-02,0,Sell
2015-02-09,9.450000,10.400000,9.400000,10.400000,10.400000,1257,10.400000,10.2500,10.243000,0.0125,5.722046e-08,1,Buy
2015-02-10,9.450000,9.450000,9.450000,9.450000,9.450000,70,10.400000,10.2750,10.254000,0.0100,-6.999912e-03,0,Sell
2015-02-11,10.350000,10.350000,10.350000,10.350000,10.350000,100,9.450000,10.2525,10.244000,-0.0050,7.000103e-03,1,Buy
2015-05-15,13.300000,13.300000,13.300000,13.300000,13.300000,0,13.300000,13.1325,12.981000,-0.1725,-3.100000e-02,0,Sell
2015-07-15,12.500000,12.500000,12.500000,12.500000,12.500000,0,12.500000,12.2125,12.168000,0.4650,5.900002e-02,1,Buy


In [5]:
# required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]

In [9]:
bt = Backtest()
capital = 500000
stocks = pd.read_csv('.\ind_nifty100list.csv')
scripts = stocks["Symbol"]
# scripts=["IEX"]
for stock in scripts:
    data = GoldenCrossverSignal(stock,0,-1)
    
    try :
        required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]
        for index, data in required_df.iterrows():
            if(data.Position == 'Buy'):
                qty = capital // data.Open
                bt.buy(stock, index, data.Open, qty)
            else:
                bt.sell(index, data.Open, 'Exit Trigger', 0)
    except :
        pass

In [10]:
bt.backtesting

,Equity Name,Trade,Entry Time,Entry Price,Exit Time,Exit Price,Quantity,Position Size,PNL,% PNL
0,ABB,Long Closed,2014-02-05,575.20,2015-03-11,1333.00,869.0,499848.811,658528.20,131.745
1,ABB,Long Closed,2015-03-19,1280.00,2015-04-22,1370.00,390.0,499200.000,35100.00,7.031
2,ABB,Long Closed,2015-05-14,1348.90,2015-06-09,1263.15,370.0,499093.009,-31727.50,-6.357
3,ABB,Long Closed,2015-06-22,1336.00,2015-08-10,1383.00,374.0,499664.000,17578.00,3.518
4,ABB,Long Closed,2015-11-06,1243.00,2015-11-09,1231.00,402.0,499686.000,-4824.00,-0.965
...,...,...,...,...,...,...,...,...,...,...
2005,ZYDUSLIFE,Long Closed,2021-03-26,423.00,2021-07-14,645.00,1182.0,499986.000,262404.00,52.482
2006,ZYDUSLIFE,Long Closed,2022-05-12,331.85,2022-05-30,376.00,1506.0,499766.109,66489.90,13.304
2007,ZYDUSLIFE,Long Closed,2022-06-22,345.00,2022-07-20,359.80,1449.0,499905.000,21445.20,4.290
2008,ZYDUSLIFE,Long Closed,2022-07-28,350.50,2023-09-20,645.00,1426.0,499813.000,419957.00,84.023


In [7]:
# bt.backtesting.to_csv('RELIANCE_TEST.csv')

In [11]:
bt.stats()

+--------------------------+--------------+
| Parameters               | Values       |
|--------------------------+--------------|
| Total Trade Scripts      | 101          |
| Total Trade              | 2010         |
| PNL                      | 230321322.37 |
| Winners                  | 1806         |
| Losers                   | 204          |
| Win Ratio                | 89.85%       |
| Total Profit             | 232766769.77 |
| Total Loss               | -2445447.4   |
| Average Loss per Trade   | -11987.49    |
| Average Profit per Trade | 128885.25    |
| Average PNL Per Trade    | 114587.72    |
| Risk Reward              | 1:10.75      |
+--------------------------+--------------+


In [201]:
required_df.to_csv("reliance_req.csv")

PermissionError: [Errno 13] Permission denied: 'reliance_req.csv'

In [ ]:
data.head()

Open         615.950012
High         625.549988
Low          615.049988
Close             620.5
Adj Close         620.5
Name: 2023-09-04 00:00:00, dtype: object